<a href="https://colab.research.google.com/github/Aliaksandr-Borsuk/Recommender_Systems_project/blob/main/notebooks/04_01_linear_models_EASE_CLEAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка


**Цель:**
- Реализовать, настроить и оценить модель EASE (Embarrassingly Shallow Autoencoders).


**Данные:**
- используем train(разбиение на v_train, val), test  из предыдущих ноутбуков.
- вход - бинаризованная пользователь–айтем матрица (implicit feedback)


## 01. Клонируем репозиторий. Подключаем GoogleDrive.

In [ ]:
!rm -rf /content/Recommender_Systems_project
!git clone https://github.com/Aliaksandr-Borsuk/Recommender_Systems_project

Cloning into 'Recommender_Systems_project'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 126 (delta 61), reused 51 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (126/126), 632.03 KiB | 3.45 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [ ]:
# подключаем диск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 8.1 MB/s eta 0:00:00


## 02. Импорты

In [ ]:
import sys
sys.path.append("/content/Recommender_Systems_project/src")

import optuna
from optuna.samplers import TPESampler

import time
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from pprint import pprint

from scipy.sparse import csr_matrix, load_npz
from sklearn.linear_model import Ridge

# Внутренние модули
from recommender.data_io import train_test_reader                 # для чтения сохранённых из 001_data_and_eda_1m_proba
from recommender.splitters import df_time_split                   # для time_split разбиения df
from recommender.preprocessing import prepare_ui_matrix           # для получения матрицы взаимодействий
from recommender.metrics import hitrate_at_k, coverage_at_k, precision_at_k,\
                     recall_at_k, ndcg_at_k, map_at_k, model_evaluation
from recommender.results_logger import save_experiment_results    # для сохранения результатов

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

DATA = Path("/content/drive/MyDrive/Colab Notebooks/data/")
RAW_DATA = DATA/"raw"
PROCESSED = DATA / "processed"
RESULTS_DIR = DATA / "results"
DATA.mkdir(exist_ok=True)

TOP_N = 10

## 03. Грузим train, test, meta_данные.



In [ ]:
train_tast_path = '/content/drive/MyDrive/Colab Notebooks/data/processed/251021_173655'

train, test, meta = train_test_reader(train_tast_path)
pprint(meta, width=80, compact=False)
print(f'\ntrain shape : {train.shape}')
print(f'test shape  : {test.shape}')
print( '\n', '*'*50, '\ntrain.head')
display(train.head(3))
print('\n', '*'*50, '\ntest.head')
display(test.head(3))

{'columns': ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'genres'],
 'created_at': '2025-10-21T17:37:00.607645',
 'min_test_interactions': 10,
 'min_train_interactions': 5,
 'n_items': 3662,
 'n_test_users': 836,
 'n_train_users': 5392,
 'test_shape': [94842, 6],
 'time_treshold': '2000-12-02T14:52:18',
 'train_shape': [800142, 6]}

train shape : (800142, 6)
test shape  : (94842, 6)

 ************************************************** 
train.head


,user_id,item_id,rating,timestamp,title,genres
0,635,1251,4,975768620,8 1/2 (1963),Drama
1,635,3948,4,975768294,Meet the Parents (2000),Comedy
2,635,1270,4,975768106,Back to the Future (1985),Comedy|Sci-Fi



 ************************************************** 
test.head


,user_id,item_id,rating,timestamp,title,genres
0,635,3789,5,975768788,"Pawnbroker, The (1965)",Drama
1,635,2987,5,979141847,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
2,635,2988,4,975769007,Melvin and Howard (1980),Drama


## 04. Бьем Train на v_train и val для подбора гиперпараметров
Используем опять вариант **B** - разбиение по времени(Time-based split).
 Все события до порога в v_train, события после в val. Это гарантирует, что ни одно событие из "будущего" не попадёт в v_train.

In [ ]:
# Бьем Train на v_train и val
# колонки для сохранения
columns_to_save = ['user_id', 'item_id', 'rating', 'timestamp', 'title', 'genres']
user_id = 'user_id'
item_id = 'item_id'
time_column = 'timestamp'

# минимальное тёплое число оценок
min_n_reitings = 5
# min кол-во оценок у каждого юзера в train и в test
n, k = 15, 10
quantile = 0.85    # квантиль разбиения
# разбиение
v_train, val = df_time_split(train, time_column, columns_to_save,
                             user_id , item_id,
                             min_n_reitings, n, k , quantile)

users_intersection = set(val['user_id']) & set(v_train['user_id'])
print('*'*70)
print(f"\n{len(users_intersection)} юзера встречаются одновременно и в v_train и в val")

# залистим тестовых пользователей
val_users = sorted(users_intersection)

print('\ntrain ')
display(v_train.head(2))
print('\ntest ')
display(val.head(2))

Порог разбиения по времени 974814617
Размеры после разбиения: train 680122 test 120020

Осталось пользователей в train : 4748
Новый размер train: (680108, 6)
Осталось пользователей в test: 211
Новый размер test: (26076, 6)
**********************************************************************

211 юзера встречаются одновременно и в v_train и в val

train 


,user_id,item_id,rating,timestamp,title,genres
0,1272,720,5,974814383,Wallace & Gromit: The Best of Aardman Animatio...,Animation
1,1272,2993,2,974814617,Thunderball (1965),Action



test 


,user_id,item_id,rating,timestamp,title,genres
0,1272,2987,4,974822814,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
1,1272,2989,4,974814713,For Your Eyes Only (1981),Action


## 05.Подготовка  матрицы и словарей
Используем то же , что и в KNN/ALS.

In [ ]:
input_dir = PROCESSED/"artifacts"

# Загрузка матрицы взаимодействий
v_train_matrix = load_npz(input_dir / "v_train_matrix.npz")
# train_matrix = load_npz(input_dir / "train_matrix.npz")

# Загрузка словарей
with open(input_dir / "user2index_v.pkl", "rb") as f:
    user2index_v = pickle.load(f)

with open(input_dir / "item2index_v.pkl", "rb") as f:
    item2index_v = pickle.load(f)

with open(input_dir / "index2user_v.pkl", "rb") as f:
    index2user_v = pickle.load(f)

with open(input_dir / "index2item_v.pkl", "rb") as f:
    index2item_v = pickle.load(f)

assert isinstance(v_train_matrix, csr_matrix), "train_matrix должен быть csr_matrix"

In [ ]:
# заменяем реальные ID на индексы из матрицы обучения модели
val_mapped = val.assign(
    user_id = val["user_id"].map(user2index_v),
    item_id = val["item_id"].map(item2index_v)
)
assert val_mapped.isna().sum().sum() == 0, 'Achtung!!! Неизвестные пользователи или айтемы!!!'

# группируем
val_items = val_mapped.groupby('user_id')['item_id'].apply(set).to_dict()

# all_items для coverage
all_items = set(v_train['item_id'].map(item2index_v).dropna().astype(int).unique())

# список пользователей для валидации
val_users_index = list(val_items.keys())

## 06. Реализация EASE (closed-form)

In [ ]:
import numpy as np
import numbers
import scipy.sparse as sp

class PET_EASE:
    """
    Реализация EASE (Steck, 2019).
    Методы:
      .fit(train_matrix, reg_lambda)
      .recommend_one(user_id, N)
      .recommend_batch_matrix(user_ids, N)
      .recommend(users_id, N)
    """

    def __init__(self, reg_lambda=500.0, clipping=False):
        """
        reg_lambda: float — регуляризация λ
        clipping: bool — обрезать отрицательные значения в B
        """
        self.reg_lambda = reg_lambda
        self.clipping = clipping
        self.B = None
        self.train_matrix = None

    def fit(self, train_matrix, reg_lambda = None):
        """
        Обучение EASE.
        train_matrix: csr_matrix (users × items)
        self.reg_lambda можно переопределять
        """
        if reg_lambda is not None:
          self.reg_lambda = reg_lambda

        self.train_matrix = train_matrix.tocsr(copy=True)
        G = (train_matrix.T @ train_matrix).toarray().astype(np.float64)

        # регуляризация
        d = np.arange(G.shape[0])
        G[d, d] += self.reg_lambda

        # инверсия
        P = np.linalg.inv(G)

        # формула EASE: B_ij = -P_ij / P_ii
        B = -P / np.diag(P)[:, None]

        np.fill_diagonal(B, 0.0)

        if self.clipping:
            B = np.maximum(B, 0.0)

        self.B = B

    def recommend_one(self, user_id, N=10):
        """
        Рекомендации для одного пользователя.
        """
        u = self.train_matrix[user_id].toarray().ravel()
        scores = u @ self.B

        # исключаем просмотренные
        scores[u > 0] = -np.inf

        # топ-N
        top_idx = np.argpartition(-scores, N)[:N]
        top_scores = scores[top_idx]
        order = np.argsort(-top_scores)
        return top_idx[order], top_scores[order]

    def recommend_batch_matrix(self, user_ids, N=10):
        """
        Рекомендации для пула пользователей.
        Возвращает:
          items  – shape (len(user_ids), N)
          scores – shape (len(user_ids), N)
        """
        U = self.train_matrix[user_ids].toarray()   # batch × items
        scores = U @ self.B                         # batch × items

        # исключаем просмотренные
        scores[U > 0] = -np.inf

        # топ-N построчно
        top_items = np.argpartition(-scores, N, axis=1)[:, :N]
        top_scores = np.take_along_axis(scores, top_items, axis=1)

        order = np.argsort(-top_scores, axis=1)
        items_sorted = np.take_along_axis(top_items, order, axis=1)
        scores_sorted = np.take_along_axis(top_scores, order, axis=1)

        return items_sorted, scores_sorted

    def recommend(self, users_id, N=10):
        """
        Универсальный метод: один пользователь или список.
        """
        if isinstance(users_id, numbers.Number):
            return self.recommend_one(users_id, N)
        elif isinstance(users_id, list) or isinstance(users_id, np.ndarray):
            return self.recommend_batch_matrix(users_id, N)
        else:
            raise ValueError("users_id должен быть int или list[int]")


## 07. подбор гиперпараметров с Optuna


In [ ]:
# подбор гиперпараметров Optuna
def make_objective_ease(val_items, X_ui, metric_fn=ndcg_at_k, k_eval=10):
    # список пользователей для валидации
    val_users_index = list(val_items.keys())

    def objective(trial):
        reg_lambda = trial.suggest_float("lambda", 20, 1e6, log=True)
        clipping = trial.suggest_categorical("clipping", [True, False])

        model = PET_EASE(reg_lambda, clipping)

        model.fit(X_ui)

        items, it_scores = model.recommend(val_users_index,  N=k_eval)
        items = items.tolist()
        recs = {val_users_index[i]: items[i] for i in range(len(val_users))}

        score = metric_fn(recs, val_items, k=k_eval)
        return score
    return objective

In [ ]:
n_trials = 30
# замерим время
start = time.time()

sampler = optuna.samplers.TPESampler(seed=RANDOM_STATE)

study_ease = optuna.create_study(direction="maximize", sampler=sampler)
study_ease.optimize(make_objective_ease(val_items, v_train_matrix), n_trials=n_trials)

end = time.time()
study_item_time = end - start

print('#'*70)
print("\nЛучшие параметры EASE:", study_ease.best_params)
print("Лучший NDCG@10:", study_ease.best_value)

print('\n')
print('*'*70)
print(f"Время выполнения: {study_item_time:.2f} секунд")

[I 2025-12-11 17:42:30,315] A new study created in memory with name: no-name-8fc79697-a682-43dc-b633-0f31220548c6
[I 2025-12-11 17:42:38,235] Trial 0 finished with value: 0.35836615098078517 and parameters: {'lambda': 374552.6280449855, 'clipping': True}. Best is trial 0 with value: 0.35836615098078517.
[I 2025-12-11 17:42:43,927] Trial 1 finished with value: 0.3566675355879872 and parameters: {'lambda': 598666.5110273527, 'clipping': True}. Best is trial 0 with value: 0.35836615098078517.
[I 2025-12-11 17:42:50,276] Trial 2 finished with value: 0.35914862144874266 and parameters: {'lambda': 58102.450495956095, 'clipping': True}. Best is trial 2 with value: 0.35914862144874266.
[I 2025-12-11 17:42:57,715] Trial 3 finished with value: 0.35671961678190556 and parameters: {'lambda': 708078.4163444896, 'clipping': False}. Best is trial 2 with value: 0.35914862144874266.
[I 2025-12-11 17:43:03,632] Trial 4 finished with value: 0.35609349077747815 and parameters: {'lambda': 832445.9919476057

######################################################################

Лучшие параметры EASE: {'lambda': 108726.80709039961, 'clipping': False}
Лучший NDCG@10: 0.3611430019248964


**********************************************************************
Время выполнения: 205.72 секунд


In [ ]:
# optuna визуализация
print("OPTUNA for PET_EASE")
optuna.visualization.plot_optimization_history(study_ease).show()
optuna.visualization.plot_param_importances(study_ease).show()
optuna.visualization.plot_parallel_coordinate(study_ease).show()

OPTUNA for PET_EASE


###Выводы:
**Optuna - молодец**

- помогла подобрать параметры, при которых EASE показывает высокий результат  на NDCG@10

**Лучшие параметры**

- lambda ~ 108726.81

- clipping ~ False

- Score ~ 0.3611 по NDCG@10

Оптимум достигается при очень сильной регуляризации и отключённой обрезке отрицательных весов. Модель работает стабильно и даёт высокие результаты без итераций.


---
**Роль параметров**  
**lambda**

- Ключевой параметр модели.

- Оптимум в диапазоне от 50k до 150k.

- При малых значениях ( < 10K ) модель переобучается на шум и теряет качество.

- Большие значения сглаживают веса и стабилизируют рекомендации.

- Важность по Optuna = 1.00 - критически влияет на метрику.

**clipping**

- Обрезка отрицательных весов не влияет на NDCG@10 ( важность < 0.01 ).

- Optuna выбрала clipping=False, то есть модель использует как положительные, так и отрицательные связи.

- Возможно, clipping влияет на разнообразие, но не на ранжирование.

**Общие наблюдения**
- EASE быстро выходит на плато качества - уже к 10‑му трейлу метрика стабилизируется.

- Модель устойчива к шуму при правильной λ, не требует итераций и обучается за секунды.

- При λ $\rightarrow \infty^~$ модель приближается к Most Popular, при λ $\rightarrow$ 0 - к переобучению.

- Высокие значения NDCG достигаются только при λ > 50k, независимо от clipping.


## 08. Получение метрик на test

In [ ]:
# Получение метрик на test
# загрузка
input_dir = PROCESSED/"artifacts"

# Загрузка матрицы взаимодействий
train_matrix = load_npz(input_dir / "train_matrix.npz")

# Загрузка словарей
with open(input_dir / "user2index.pkl", "rb") as f:
    user2index = pickle.load(f)

with open(input_dir / "item2index.pkl", "rb") as f:
    item2index = pickle.load(f)

with open(input_dir / "index2user.pkl", "rb") as f:
    index2user = pickle.load(f)

with open(input_dir / "index2item.pkl", "rb") as f:
    index2item = pickle.load(f)

assert isinstance(train_matrix, csr_matrix), "train_matrix должен быть csr_matrix"
train_matrix

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 800142 stored elements and shape (5392, 3662)>

In [ ]:
# заменяем реальные ID на индексы
test_mapped = test.assign(
    user_id = test["user_id"].map(user2index),
    item_id = test["item_id"].map(item2index)
)
assert test_mapped.isna().sum().sum() == 0, 'Achtung!!! Неизвестные пользователи или айтемы!!!'

# группируем
test_items = test_mapped.groupby('user_id')['item_id'].apply(set).to_dict()

# all_items
all_items = set(train['item_id'].map(item2index).dropna().astype(int).unique())

In [ ]:
# Вспомним лучшее:
print('Лучшие парамеры EASE :',study_ease.best_params)

Лучшие парамеры EASE : {'lambda': 108726.80709039961, 'clipping': False}


In [ ]:
best = study_ease.best_params
test_users = list(test_items.keys())
pet_ease = PET_EASE(best['lambda'], best['clipping'])


pet_ease.fit(train_matrix)
# Рекомендации
items, it_scores = pet_ease.recommend(test_users,  N=TOP_N)
items = items.tolist()
ease_recs = {test_users[i]: items[i] for i in range(len(test_users))}

ease_results = model_evaluation(
    ease_recs, test_items, all_items, TOP_N,
    f'ease_lambda={best['lambda']:.0f}'
)
ease_results

,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10
ease_lambda=108727,0.838517,0.338517,0.042712,0.356757,0.243356,0.045603


## 09. Сохраняем результаты

In [ ]:
# Сохраняем результаты
results_data, json_file, csv_file = save_experiment_results(
    result = ease_results,
    model_name = f'ease_lambda={best['lambda']:.0f}',
    meta=meta,
    results_dir=RESULTS_DIR
)

Результат добавлен в существующий CSV файл
JSON результат сохранен как: ease_lambda=108727_20251211_185547.json
CSV со всеми экспериментами: all_experiments_results.csv
Все результаты в: /content/drive/MyDrive/Colab Notebooks/data/results

СВОДКА ЭКСПЕРИМЕНТА
Модель: ease_lambda=108727
Метка времени: 20251211_185547
Дата оценки: 2025-12-11T18:55:47
Размер train: 800,142
Размер test: 94,842
Пользователей в test: 836
Уникальных предметов: 3662
HitRate@10: 83.9%
precision@10: 33.85%
recall@10: 4.27%
ndcg@10: 35.68%
map@10: 24.34%
Coverage@10: 4.56%

Последние эксперименты (6 всего):


,model_name,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10,timestamp,evaluation_date
1,itemKNN_tfidf_k=312,0.812201,0.329067,0.040669,0.345263,0.233586,0.074823,20251108_084840,2025-11-08T08:48:40.783074
2,userKNN_bmp25_k=991,0.836124,0.341029,0.043678,0.361278,0.244691,0.063900,20251108_084943,2025-11-08T08:49:43.731391
3,trancated_svd_n_comp=5_n_iter=42,0.851675,0.348206,0.045528,0.365538,0.247310,0.091207,20251116_190936,2025-11-16T19:09:36.863512
4,als_factors=5_iter=21_alpha=0.6_reg=0.02,0.840909,0.340191,0.043517,0.358804,0.244510,0.095576,20251210_110349,2025-12-10T11:03:49.859474
5,ease_lambda=108727,0.838517,0.338517,0.042712,0.356757,0.243356,0.045603,20251211_185547,2025-12-11T18:55:47.668555


# Итого:


**наблюдения:**
**ease, λ~108k, clipping=False**

- по качеству ранжирования близка к ALS и UserKNN, немного уступает Truncated SVD

- **HitRate@10:** 83.9%

- **Precision@10:** 33.85%

- **Recall@10:** 4.27%

- **NDCG@10:** 35.68%

- **MAP@10:** 24.34%

**Однако:**

- **Coverage@10**: 4.56% — заметно ниже, чем у ALS (9.6%) и SVD (9.1%). Это говорит о том, что EASE концентрируется на более популярных и «сильных» связях между айтемами, жертвуя разнообразием каталога.

## **Выводы:**
**Truncated SVD** остаётся лидером по качеству ранжирования, что ожидаемо: она использует явные рейтинги и градуированную информацию.

**EASE** показывает результат на уровне **ALS** и **UserKNN** по точности, но проигрывает по **Coverage**. Это отражает природу модели: она строит глобальную линейную регрессию между айтемами, которая хорошо улавливает сильные корреляции, но хуже покрывает длинный хвост каталога.

**ALS** остаётся лучшим компромиссом между точностью и разнообразием: чуть слабее по **NDCG**, но значительно шире по охвату.

**KNN‑модели** конкурентоспособны, особенно **UserKNN**, но требуют хранения полной матрицы схожести и хуже масштабируются.

Итог: **EASE** — сильный кандидат, если важна простота, скорость обучения (закрытая форма, без итераций) и высокая точность ранжирования на популярных айтемах. Однако, если приоритет — разнообразие каталога и покрытие, то **ALS** выглядит предпочтительнее. Низкий **Coverage** у **EASE** показывает, что модель склонна рекомендовать «популярные кластеры» и менее эффективна для увеличения разнообразия.

**Таким образом:**

**SVD** — лидер по качеству.

**ALS** — лидер по балансу качества и разнообразия.

**EASE** — лидер по простоте и скорости, но с ограниченным охватом каталога.

# P.S.
Как замечено ранее - очень больщая λ толкает модель к снижению разнообразия. Ослабим ка жёсткую хватку регуляризации.

In [ ]:
# уменьшим lambda на порядок
pet_ease = PET_EASE(10000, best['clipping'])


pet_ease.fit(train_matrix)
# Рекомендации
items, it_scores = pet_ease.recommend(test_users,  N=TOP_N)
items = items.tolist()
ease_recs = {test_users[i]: items[i] for i in range(len(test_users))}

ease_results = model_evaluation(
    ease_recs, test_items, all_items, TOP_N,
    f'ease_lambda={10000:.0f}'
)
ease_results

,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10
ease_lambda=108727,0.850478,0.342703,0.046752,0.362255,0.240761,0.114418


In [ ]:
# уменьшим lambda и ещё на порядок
pet_ease = PET_EASE(1000, best['clipping'])


pet_ease.fit(train_matrix)
# Рекомендации
items, it_scores = pet_ease.recommend(test_users,  N=TOP_N)
items = items.tolist()
ease_recs = {test_users[i]: items[i] for i in range(len(test_users))}

ease_results = model_evaluation(
    ease_recs, test_items, all_items, TOP_N,
    f'ease_lambda={1000:.0f}'
)
ease_results

,hit_rate@10,precision@10,recall@10,ndcg@10,map@10,coverage@10
ease_lambda=1000,0.822967,0.315191,0.042639,0.336882,0.217801,0.206171


## **Коротенький Вывод:**

**λ выступает как «ручка регулятора баланса»:**

- большой λ - устойчивость и популярность, но низкая диверсификация

- малый λ - больше разнообразия и охвата, но выше риск шума